In [ ]:
import torch as T
import numpy as np

from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

from IPython import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set()

In [ ]:
def func(x):
    return np.sin(x) * np.cos(0.5 * x)

X = np.random.rand(100_000) * 2 * np.pi
# X = np.linspace(-10, 10, 100_000)
Y = func(X)

plt.figure(figsize=(14,5))
plt.scatter(X, Y, s=0.05)

In [ ]:
class FuncApprox(nn.Module):
    def __init__(self, num_in, num_out):
        super(FuncApprox, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_in, 1024), nn.ReLU(inplace=True),
            nn.Linear(1024, 1024), nn.ReLU(inplace=True),
            nn.Linear(1024, num_out))
        
    def forward(self, x):
        return self.model(x)

In [ ]:
def test(model):
    def get(xs):
        with T.no_grad():
            return model(T.tensor([xs]).T.float().cuda()).cpu()

#     xs = np.random.rand(10_000) * 2 * np.pi
    xs = X
    ys = get(xs)

    plt.scatter(xs, ys, s=0.5, c='r', label='approx')
    plt.scatter(xs, func(xs), s=0.5, c='g', alpha=0.7, label='true')
    plt.legend()

In [ ]:
BATCH_SIZE = 1024
LR = 1e-3
NUM_EPOCHS = 100

X_train, X_val, y_train, y_val = map(T.tensor, train_test_split(X, Y, test_size=0.2))

train_dl = DataLoader(TensorDataset(X_train.unsqueeze(1), y_train.unsqueeze(1)), batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(TensorDataset(X_val.unsqueeze(1), y_val.unsqueeze(1)), batch_size=BATCH_SIZE, shuffle=True)

model = FuncApprox(num_in=1, num_out=1)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss(reduction="sum")

model.cuda()

for epoch in tqdm(range(1, NUM_EPOCHS+1)):
    model.train()
    tl = 0

    for X_train, y_train in train_dl:
        X_train = X_train.type(T.float32).cuda()
        y_train = y_train.type(T.float32).cuda()

        optimizer.zero_grad()

        y_hat = model(X_train)
        loss = criterion(input=y_hat, target=y_train)
        loss.backward()

        tl += loss.item() / len(train_dl)

        optimizer.step()

    model.eval()
    vl = 0

    for X_val, y_val in val_dl:
        X_val = X_val.type(T.float32).cuda()
        y_val = y_val.type(T.float32).cuda()

        y_hat = model(X_val)

        loss = criterion(input=y_hat, target=y_val)
        vl += loss.item() / len(val_dl)
        
    done = vl <= 0.18

    if True:#epoch % (NUM_EPOCHS//10) == 0:
        print(f"epoch {epoch:3d} / {NUM_EPOCHS:3d} | train loss = {tl:,.12f} | val loss = {vl:,.12f}")
        
        plt.figure(figsize=(8,4))
        test(model)
        plt.show()
        
        if epoch != NUM_EPOCHS and not done:
            display.clear_output(wait=True)
            
    if done:
        break